# Supervised

This notebook extracts respiration signals, for each subject in the dataset, with the following models:
- MTTS-CAN
- BigSmall

In [ ]:
import os
import numpy as np
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()
scenarios = dataset.get_scenarios(['101_natural_lighting'])
scenarios

## Conduct experiments

In [ ]:
from datetime import datetime

manifest = {
    'timestamp_start': datetime.now(),
    'scenarios': scenarios,
}

In [ ]:
import respiration.utils as utils
import respiration.extractor.mtts_can as mtts_can
import respiration.extractor.big_small as big_small

device = utils.get_torch_device()

frame_depth = 10
mtts_model = mtts_can.load_model(frame_depth=frame_depth)
big_small_model, _ = big_small.load_model(device=device)

extracted_signals = []

# TODO: Use batch processing
for (subject, setting) in scenarios:
    print(f'Processing {subject} - {setting}')

    frames, meta = dataset.get_video_rgb(subject, setting)

    #
    # MTTS-CAN model to extract the signal
    #

    mtts_start = datetime.now()

    resized, normalized = mtts_can.preprocess_video_frames(frames)
    cutoff = mtts_can.calculate_cutoff(resized.shape[0], frame_depth)

    resized = resized[:cutoff]
    normalized = normalized[:cutoff]

    mtts_raw = mtts_model.predict(
        (resized, normalized),
        batch_size=100
    )
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'mtts_can',
        'duration': datetime.now() - mtts_start,
        'sampling_rate': meta.fps,
        'signal': np.cumsum(mtts_raw[1]).tolist(),
    })

    #
    # Use the BigSmall model to extract the signal
    #

    big_small_start = datetime.now()
    big_small_raw = big_small.batch_process(big_small_model, frames, device)
    extracted_signals.append({
        'subject': subject,
        'setting': setting,
        'model': 'big_small',
        'duration': datetime.now() - big_small_start,
        'sampling_rate': meta.fps,
        'signal': big_small_raw.tolist(),
    })

    # Garbage collect the frames
    del frames

In [ ]:
manifest['timestamp_finish'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
output_dir = utils.dir_path('outputs', 'signals', mkdir=True)

# Save the evaluation dataframe
csv_path = os.path.join(output_dir, 'pretrained_predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(output_dir, 'pretrained_manifest.json')
utils.write_json(json_path, manifest)